In [31]:
import os

In [32]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\gemstonePricePrediction'

In [33]:
os.chdir('c:\\Users\\quamr\\OneDrive\\Desktop\\project\\gemstonePricePrediction')

In [34]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\gemstonePricePrediction'

In [35]:
import pandas as pd

In [36]:
df = pd.read_csv("artifacts/data_ingestion/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       193573 non-null  int64  
 1   carat    193573 non-null  float64
 2   cut      193573 non-null  object 
 3   color    193573 non-null  object 
 4   clarity  193573 non-null  object 
 5   depth    193573 non-null  float64
 6   table    193573 non-null  float64
 7   x        193573 non-null  float64
 8   y        193573 non-null  float64
 9   z        193573 non-null  float64
 10  price    193573 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 16.2+ MB


In [38]:
df.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [39]:
df.shape

(193573, 11)

In [40]:
df.columns

Index(['id', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y',
       'z', 'price'],
      dtype='object')

In [41]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [42]:
from src.gemstonePricePrediction.constants import *
from src.gemstonePricePrediction.utils.common import read_yaml, create_directories

In [43]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath=CONFIG_FILE_PATH, 
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_validation_config(self) -> DataValidationConfig:
      config = self.config.data_validation
      schema = self.schema.COLUMNS

      create_directories([config.root_dir])

      data_validation_config = DataValidationConfig(
        root_dir=config.root_dir,
        STATUS_FILE=config.STATUS_FILE,
        unzip_data_dir=config.unzip_data_dir,
        all_schema=schema

      )

      return data_validation_config

In [44]:
import os
from src.gemstonePricePrediction.logger import logging

In [45]:
class DataValidation:
  def __init__(self,config: DataValidationConfig):
    self.config = config

  def validate_all_columns(self) -> bool:
    try:
      validation_status = None
      
      data = pd.read_csv(self.config.unzip_data_dir)
      all_cols = list(data.columns)

      all_schema = self.config.all_schema.keys()

      for col in all_cols:
        if col not in all_schema:
          validation_status = False
          with open(self.config.STATUS_FILE, 'w') as f:
            f.write(f"validation status: {validation_status}")

        else:
          validation_status = True
          with open(self.config.STATUS_FILE, 'w') as f:
            f.write(f"validation status : {validation_status}")
      return validation_status

    except Exception as e:
      raise e

In [46]:
try:
  config = ConfigurationManager()
  data_validation_config = config.get_data_validation_config()
  data_validation = DataValidation(config = data_validation_config)
  data_validation.validate_all_columns()

except Exception as e:
  raise e

[2026-02-10 18:53:55,649: INFO: common: yaml file loaded successfully from: C:\Users\quamr\OneDrive\Desktop\project\gemstonePricePrediction\config\config.yaml]
[2026-02-10 18:53:55,658: INFO: common: yaml file loaded successfully from: C:\Users\quamr\OneDrive\Desktop\project\gemstonePricePrediction\params.yaml]
[2026-02-10 18:53:55,663: INFO: common: yaml file loaded successfully from: C:\Users\quamr\OneDrive\Desktop\project\gemstonePricePrediction\schema.yaml]
[2026-02-10 18:53:55,665: INFO: common: created directory at: artifacts]
[2026-02-10 18:53:55,668: INFO: common: created directory at: artifacts/data_validation]
